In [1]:
# Instale as bibliotecas necessárias
#!pip install dash plotly pandas jupyter-dash
#!pip install dash
#!pip install pandas_market_calendars

In [18]:
# Importe as bibliotecas necessárias

from dash import Dash, html, dcc, Input, Output, dash_table
from jupyter_dash import JupyterDash
JupyterDash.infer_jupyter_proxy_config()
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
#import pandas_market_calendars as mcal

import warnings
warnings.filterwarnings('ignore')

In [19]:
# Lendo os arquivos xlsx

preLast = pd.read_excel("dados/Risco_Retorno_Base.xlsx",sheet_name=2,index_col = 0, header = 1)
Portfolios = pd.read_excel("dados/Risco_Retorno_Base.xlsx",sheet_name=1,index_col = None)
Volume = pd.read_excel("dados/Risco_Retorno_Base.xlsx",sheet_name=3,index_col = None)

In [34]:
preLast['Data']

KeyError: 'Data'

In [29]:
# Entrada de Variáveis

lista_1 = Portfolios.columns.tolist()
lista_2 = [preLast.columns[-1], preLast.columns[-2]]
print("-----------------------------------------")
print(f"Tipos de carteiras: {', '.join(lista_1)}")
print("")
selecao_carteira = input("Selecione a carteira:  ")
print("")
print("-----------------------------------------")
data_inicial = str(input("Insira a data de aquisição (formato YYYY-MM-DD): "))
print("")
print("-----------------------------------------")
valor_inicial = float(input("Insira o valor inicial da carteira: "))
print("")
print("-----------------------------------------")
print("Insira o período em dias para o controle de posição (ADTV).")
periodo = int(input("Período: "))

# Data final (para o trabalho será considerada 2023-03-31)
data_final = "2023-03-31"

# Data final para último dia útil
#cal = mcal.get_calendar('B3')
#today = date.today()
#last_business_day = cal.schedule(start_date=today - timedelta(days=365), end_date=today).index[-1].date()
#data_final = last_business_day.strftime('%Y-%m-%d')

-----------------------------------------
Tipos de carteiras: Blue Chips, Mid Caps, Small Caps, Micro Caps, High Return, Low Return, High Volatility, Low Volatility, High Liquidity, Low Liquidity, High Beta, Market Beta, Low Beta, High Alpha, Low Alpha, High Sharpe, Low Sharpe, High P/E, Low P/E, High P/B, Low P/B, High Debt, Low Debt, Banks

Selecione a carteira:  High Beta

-----------------------------------------
Insira a data de aquisição (formato YYYY-MM-DD): 2019-12-31

-----------------------------------------
Insira o valor inicial da carteira: 10000

-----------------------------------------
Insira o período em dias para o controle de posição (ADTV).
Período: 60


In [30]:
# Organizando o Dataframe Volume

# Remover os nomes das colunas
Volume.columns = ['' for c in Volume.columns]

# usar a primeira linha como nome das colunas
Volume.columns = Volume.iloc[0]

# remover a primeira linha
Volume= Volume[1:]

# converter a coluna "Data" para o tipo "datetime"
Volume['Data'] = pd.to_datetime(Volume['Data'], format='%Y-%m-%d')

# Transformando a coluna data no índice
Volume = Volume.set_index('Data')

KeyError: 'Data'

In [31]:
# Organizando o DataFrame preLast

# Criando um Dataframe preLast_1
preLast_1 = pd.DataFrame() # Contém o Índice Bovespa e Índice DI

# Remover as colunas .BVSP e INDEX_DI do primeiro dataframe e adicioná-las ao segundo dataframe
preLast_1.insert(loc=len(preLast_1.columns), column='.BVSP', value=preLast.pop('.BVSP'))
preLast_1.insert(loc=len(preLast_1.columns), column='DI_INDEX', value=preLast.pop('DI_INDEX'))

KeyError: '.BVSP'

## Portfólio de ativos

In [28]:
print("O portfolio escolhido: ", selecao_carteira)
print("")
print("Data de inicio: ", data_inicial)
print("")
print("Data final: ", data_final)
print("")
print("Valor inicial investido: R$",valor_inicial)

O portfolio escolhido:  Mid Caps

Data de inicio:  31-12-2019

Data final:  2023-03-31

Valor inicial investido: R$ 10000.0


In [24]:
# Preço de fechamento na data inicial e na data atual de cada ativo do portfolio
preco_inicial = preLast[Portfolios[selecao_carteira]].loc[data_inicial] 
preco_atual = preLast[Portfolios[selecao_carteira]].loc[data_final]

# Cálculo da quantidade inicial de ativos no portfólio no período de aquisição
quantidades = valor_inicial / len(Portfolios[selecao_carteira]) / preco_inicial
q = quantidades.astype(int)

# Calculo da posição inicial e final de cada ativo do porfolio
valor_posicao_inicial = round(q * preco_inicial,2)
valor_posicao_final = round(q * preco_atual,2)

# Valor total das posições nas datas iniciais e finais do portfólio
valor_total_inicial = valor_posicao_inicial.sum()
valor_total_final = valor_posicao_final.sum()

# Retorno do Portfólio
retorno_portfolio = round(((valor_total_final / valor_total_inicial)-1)*100, 2)

# Cálculo do retorno para o benchmarks (Índice Bovespa e Índice DI)
# IBOV
preco_bvsp_data_inicial = preLast_1['.BVSP'].loc[data_inicial]
preco_bvsp_data_atual = preLast_1['.BVSP'].loc[data_final]
retorno_bvsp = round(((preco_bvsp_data_atual / preco_bvsp_data_inicial)-1)*100,2)

# DI
preco_di_data_inicial = preLast_1['DI_INDEX'].loc[data_inicial]
preco_di_data_atual = preLast_1['DI_INDEX'].loc[data_final]
retorno_di = round(((preco_di_data_atual / preco_di_data_inicial)-1)*100,2)


# ADTV - Average Daily Traded Volume  
# Cria uma função 
def adtv_portfolio(preLast, Portfolios, selecao_carteira, data_inicial, data_final):
    
    # Determina o período
    data_final = pd.to_datetime(data_final)
    data_inicial = data_final - timedelta(days=periodo)
    
    # Realiza - se o loop em cada ativo que compõem o portfolio e calcula o volume médio do período
    Volume_periodo = []
    for ativo in Portfolios[selecao_carteira]:
        volume_ativo = Volume.loc[(Volume.index >= data_inicial) & (Volume.index <= data_final), ativo].fillna(0)
        Volume_periodo.append(volume_ativo)
    Volume_periodo = pd.concat(Volume_periodo, axis=1)
    adtv = round(Volume_periodo.mean(), 2)
    
    return adtv


# Índice de concentração
concentracao = round((valor_posicao_final / valor_total_inicial)*100,2)

 # ADTV do portfólio
adtv_periodo = adtv_portfolio(preLast, Portfolios, selecao_carteira, data_inicial, data_final)
adtv_percentual = round((valor_total_final / adtv_periodo)*100,2)

KeyError: '31-12-2019'

In [25]:
# Indicadores

# Calculo da série histórica do porfólio
retorno = preLast[Portfolios[selecao_carteira]].loc[data_inicial:data_final]
ret_portfolio = retorno.mul(q, axis=1)
retorno_portfolio = ret_portfolio.sum(axis=1)
retorno_carteira = np.log(retorno_portfolio / retorno_portfolio.shift(1)).dropna()
ret_carteira = sum(retorno_carteira)
returns_carteira = retorno_carteira.mean() * 252

# Calculo da cota
cota = round(retorno_portfolio / valor_total_inicial,2)

# Calculo da série histórica do IBOV e DI
# DI
rf = preLast_1["DI_INDEX"].loc[data_inicial:data_final]
rf_returns = np.log(rf / rf.shift(1)).dropna()
ret_rf = sum(rf_returns)
returns_rf = rf_returns.mean() * 252
volatility_rf = np.std(rf_returns)*np.sqrt(252)

# IBOV
rm = preLast_1[".BVSP"].loc[data_inicial:data_final]
rm_returns = np.log(rm / rm.shift(1)).dropna()
ret_rm = sum(rm_returns)
returns_ibov = rm_returns.mean() * 252
volatility_ibov = np.std(rm_returns)*np.sqrt(252)

# Calculo do Beta
covariance = np.cov(retorno_carteira, rm_returns)[0][1]
variance = np.var(rm_returns)
beta = round(covariance / variance,2)

# Correlação
corr_matrix = np.corrcoef(retorno_carteira, rm_returns)
correlacao = round(corr_matrix[0, 1],2)

# Índice Alpha
alpha_ratio = round((ret_carteira - ret_rf) - beta * (ret_rm - ret_rf),2)

# Índice Sharpe
volatilidade_carteira = np.std(retorno_carteira)*np.sqrt(252)
sharpe_ratio = round((ret_carteira - ret_rf) / volatilidade_carteira,2)

# Índice Sortino
express_returns = ret_carteira - ret_rf
returns_std = retorno_carteira[retorno_carteira<0].std()
sortino_ratio = round(express_returns / returns_std,2)

# Drawdown
cum_returns = (1 + retorno_carteira).cumprod()   
running_max = cum_returns.cummax()  
drawdown = (cum_returns - running_max) / running_max *100 

NameError: name 'q' is not defined

In [26]:
# Organizando as informações para o Dashboard

# DataFrame do Portfólio
df_portfolio = pd.DataFrame()
df_portfolio["Quant. Ativos"] = q
df_portfolio["Posição Inicial (R$)"] = valor_posicao_inicial
df_portfolio["Posição Final (R$)"] = valor_posicao_final
df_portfolio["Concentração (%)"] = concentracao
df_portfolio[" % ADTV"] = adtv_percentual
df_portfolio['ADTV'] = adtv_periodo

# DataFrame dos Indicadores
df_indicador = pd.DataFrame()
df_indicador["Retorno Portfolio"] = returns_carteira
df_indicador["Volatilidade Portfolio"] = volatilidade_carteira
df_indicador["Beta"] = beta
df_indicador["Índice Correlação"] = correlacao
df_indicador["Índice Sharpe"] = sharpe_ratio
df_indicador["Índice Alpha"] = alpha_ratio
df_indicador["Índice Sortino"] = sortino_ratio
df_indicador["Retorno IBOV"] = returns_ibov
df_indicador["Volatilidade IBOV"] = volatility_ibov
df_indicador["Retorno DI"] = returns_rf
df_indicador["Volatilidade DI"] = volatility_rf
df_indicador = df_indicador.T

# DataFrame para construção gráfica

# plot gráfico valores financeiros
df_financeiro = pd.DataFrame({
    "Valor Inicial": [valor_total_inicial],
    "Valor sem Investir": [valor_inicial - valor_total_inicial],
    "Valor Atual": [valor_total_final]
}).T
df_financeiro.reset_index(inplace=True)
df_financeiro.columns = ['Posição', 'R$']

# Plot gráfico dos retornos e drawdown
df_grafico = pd.DataFrame()
df_grafico["Portfolio"] = (1 + retorno_carteira).cumprod() - 1 # Calcula o retorno acumulativo no período diário
df_grafico["IBOV"] = (1 + rm_returns).cumprod() - 1 # Calcula o retorno acumulativo no período diário
df_grafico["DI"] = (1 + rf_returns).cumprod() - 1 # Calcula o retorno acumulativo no período diário
df_grafico["Drawdown"] = drawdown

NameError: name 'q' is not defined

## Construção do dashboard

In [189]:
# Crie o aplicativo Dash
app = JupyterDash(__name__)

# Defina o layout do aplicativo
app.layout = html.Div(children=[
    html.H1(children='Análise de Rendas Variáveis - Trabalho Prático'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),
    
    html.Div(id="texto"),

    dcc.Dropdown(
        id='lista_fundo',
        options=[
            {'label': 'Portfolio', 'value': 'Portfolio'},
            {'label': 'IBOV', 'value': 'IBOV'},
            {'label': 'DI', 'value': 'DI'}
        ],
        value='Portfolio'
    ),
    
    dcc.Graph(
        id='grafico-drawdown',
        figure=px.line(df_grafico, x=df_grafico.index, y="Drawdown")
    )
])

@app.callback(
    Output('texto', 'children'),
    [Input('lista_fundo', 'value')])
def update_output(value):
    return f'Você selecionou {value}'

# Inicie o aplicativo no modo inline
app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/



OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.